# Data Frame Aggregation

In Dataframe aggregation we will pefrom aggregate and set based opertions on dataframes.

For more information follow the below links:

1. [snowflake.snowpark.DataFrame.group_by](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.DataFrame.group_by)

2. [snowflake.snowpark.RelationalGroupedDataFrame](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/latest/snowpark/api/snowflake.snowpark.RelationalGroupedDataFrame#snowflake.snowpark.RelationalGroupedDataFrame)

3. [How to Create a Complex Query with Snowpark DataFrame in Python](https://medium.com/snowflake/how-to-create-a-complex-query-with-snowpark-dataframe-in-python-2d31b9e0961b)

In [ ]:
#Importing libraries
from snowflake.snowpark.functions import *
from snowflake.snowpark.context import get_active_session

session = get_active_session()

In [ ]:
df= session.table('SNOWPARK_DB.SOURCED.EMPLOYEES')
df

In [ ]:
res = df.agg({"salary":"sum","*":"count"})
res
res = df.agg(("salary", "sum"), ("*", "count"))
res
res = df.agg(sum("salary").alias("s"), count("*").alias("c"))
res

In [ ]:
#Group by:
res = df.group_by("department").agg({"salary": "sum", "*": "count"})
res
res = df.group_by("department").sum("salary")
res
res = df.group_by("department").function("sum")("salary")
res

In [ ]:
df.group_by("department").agg(("salary","sum"),("salary","max"))


In [ ]:
#Drop Duplicate
df.select("department").distinct()

In [ ]:
#Employees top salary per department:
df.sort("salary").drop_duplicates("department")

In [ ]:
from snowflake.snowpark import GroupingSets

gs = GroupingSets([col("department")], [col("job")])
df.group_by_grouping_sets(gs).count()


In [ ]:
from snowflake.snowpark.functions import grouping

df.group_by_grouping_sets(gs).agg([
        count("*").alias("count"),
        grouping("department", "job").alias("g")
    ]).sort("g", "department", "job")

In [ ]:
df2 = df.rollup("department", "job").count()
df2

df2 = df.rollup("department", "job").agg([
        count("*").alias("count"),
        grouping("department", "job").alias("g")
    ]).sort("g", "department", "job")
df2

In [ ]:
df2 = df.cube("department", "job").count()
df2

df2 = df.cube("department", "job").agg([
        count("*").alias("count"),
        grouping("department", "job").alias("g")
    ]).sort("g", "department", "job")
df2

In [ ]:
df2 = df.select("department", "job", "salary"
    ).filter(col("job").isin(['CLERK', 'MANAGER', 'ACCOUNTANT']))
df2

# CROSSTAB
dfc = df2.crosstab("department", "job")
dfc

In [ ]:
df3 = df2.group_by("department", "job").sum("salary")
df3

# PIVOT
dfp = df2.pivot("job", ['CLERK', 'MANAGER', 'ACCOUNTANT']).sum("salary")
dfp

# UNPIVOT
dfu = dfp.unpivot("salary", "job", ["'CLERK'", "'MANAGER'", "'ACCOUNTANT'"])
dfu